In [ ]:
import os
import re
import pickle
from typing import List
from pathlib import Path
from langchain.docstore.document import Document
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import MarkdownHeaderTextSplitter

class CommonRAGUtil:
    def __init__(self):
        pass

    def save_documents(self, docs: List[Document], output_path: str):
        """
        List[Document]를 pickle 파일로 저장합니다.
        
        Args:
            docs (List[Document]): 저장할 문서 리스트
            output_path (str): 저장할 디렉토리 경로
        """
        os.makedirs(output_path, exist_ok=True)
        docs_file_path = os.path.join(output_path, "documents.pkl")
        
        with open(docs_file_path, "wb") as f:
            pickle.dump(docs, f)
        
        print(f"✅ 문서 저장 완료: {docs_file_path}")
        print(f"📄 저장된 문서 수: {len(docs)}")


    def load_documents(self, input_path: str) -> List[Document]:
        """
        저장된 pickle 파일에서 List[Document]를 로드합니다.
        
        Args:
            input_path (str): 문서가 저장된 디렉토리 경로 또는 파일 경로
            
        Returns:
            List[Document]: 로드된 문서 리스트
        """
        # 디렉토리 경로인 경우 documents.pkl 파일을 찾음
        if os.path.isdir(input_path):
            docs_file_path = os.path.join(input_path, "documents.pkl")
        else:
            docs_file_path = input_path
        
        if not os.path.exists(docs_file_path):
            raise FileNotFoundError(f"문서 파일을 찾을 수 없습니다: {docs_file_path}")
        
        with open(docs_file_path, "rb") as f:
            docs = pickle.load(f)
        
        print(f"✅ 문서 로드 완료: {docs_file_path}")
        print(f"📄 로드된 문서 수: {len(docs)}")
        
        return docs


    def embed_and_save_with_docs(self, docs: List[Document], output_path: str, model_name: str = "bge-m3:latest"):
        """
        문서를 임베딩하고 FAISS 데이터베이스로 저장하며, 동시에 원본 문서도 저장합니다.
        
        Args:
            docs (List[Document]): 처리할 문서 리스트
            output_path (str): 저장할 디렉토리 경로
            model_name (str): 임베딩 모델명
        """
        # 임베딩 모델 초기화
        embedding_model = OllamaEmbeddings(model=model_name)
        
        # FAISS 데이터베이스 생성 및 저장
        db = FAISS.from_documents(docs, embedding_model)
        db.save_local(output_path)
        print(f"✅ 임베딩 저장 완료: {output_path}")
        
        # 원본 문서도 함께 저장
        self.save_documents(docs, output_path)
        

    def load_both_faiss_and_docs(self, folder_path: str, model_name: str = "bge-m3:latest") -> tuple[FAISS, List[Document]]:
        """
        FAISS 벡터 데이터베이스와 원본 문서를 모두 로드합니다.
        
        Args:
            folder_path (str): 데이터가 저장된 디렉토리 경로
            model_name (str): 임베딩 모델명
            
        Returns:
            tuple: (FAISS 데이터베이스, List[Document])
        """
        # 임베딩 모델 초기화
        embedding_model = OllamaEmbeddings(model=model_name)
        
        # FAISS 데이터베이스 로드
        loaded_db = FAISS.load_local(
            folder_path=folder_path,
            embeddings=embedding_model,
            allow_dangerous_deserialization=True,
        )
        print(f"✅ FAISS 데이터베이스 로드 완료: {folder_path}")
        
        # 원본 문서 로드
        docs = self.load_documents(folder_path)
        
        return loaded_db, docs